In [1]:
%load_ext autoreload
%autoreload 2

# Prepare dataset

In [2]:
from dataloader_bdd_100k import BDD_loader

In [3]:
from os import listdir

In [4]:
homedir = './datasets/bdd_100k/bdd100k/'

label_dir = homedir + 'labels/'
image_dir = homedir + 'images/100k/'
listdir(image_dir),listdir(label_dir)


(['train', 'val', 'test'],
 ['bdd100k_labels_images_train.json', 'bdd100k_labels_images_val.json'])

In [5]:
import glob
import pickle
training_labels_path = glob.glob(label_dir + '*train.json')[0]
validate_labels_path = glob.glob(label_dir + '*val.json')[0]


with open("./datalists/bdd100k_train_images_path.txt", "rb") as fp:
    training_images_list = pickle.load(fp)
with open("./datalists/bdd100k_val_images_path.txt", "rb") as fp:
    validate_images_list = pickle.load(fp)

In [6]:
bdd_dataset_train = BDD_loader(training_images_list, training_labels_path, training=True)


100%|██████████| 69863/69863 [00:01<00:00, 37432.71it/s]


In [7]:
bdd_dataset_valid = BDD_loader(validate_images_list,validate_labels_path,training=False)

100%|██████████| 10000/10000 [00:00<00:00, 38806.90it/s]


## Data loaders

In [9]:
from torch.utils.data import DataLoader
from utils import collate_fn

In [15]:
training_loader = DataLoader(
                            bdd_dataset_train,
                            batch_size=8,
                            shuffle=True,
                            num_workers=4,
                            collate_fn=collate_fn)

# Load model

In [16]:
import torch
import torchvision

In [19]:
from utils import label_dic
print(label_dic)
num_classes = len(label_dic)
print(len(label_dic))

{'bus': 0, 'traffic light': 1, 'traffic sign': 2, 'person': 3, 'bike': 4, 'truck': 5, 'motor': 6, 'car': 7, 'train': 8, 'rider': 9}
10


In [20]:
def get_faster_rcnn_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
        in_features, num_classes
    )  # replace the pre-trained head with a new one
    return model.cuda()

faster_rcnn = get_faster_rcnn_model(num_classes)

# Training faster rcnn

In [24]:
load_model = False
model_name = 'fasterrcnn_bdd100k_10class_25epoch'

if load_model:
    checkpoint = torch.load(model_name + ".pth")
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
# epoch = checkpoint['epoch']

{'bus': 0,
 'traffic light': 1,
 'traffic sign': 2,
 'person': 3,
 'bike': 4,
 'truck': 5,
 'motor': 6,
 'car': 7,
 'train': 8,
 'rider': 9}

In [ ]:
print("Training started")

num_epochs = 25

for epoch in tqdm(range(num_epochs)):
    train_one_epoch(model, optimizer, dl, device, epoch, print_freq=200)
    lr_scheduler.step()

    if epoch == 5 or epoch == 10 or epoch == 15 or epoch == 20 or epoch == 24:
        save_name = "saved_models/bdd100k_" + str(epoch) + ".pth"
        torch.save(
            {"model": model.state_dict(), "optimizer": optimizer.state_dict(),},
            save_name,
        )
        print("Saved model", save_name)